In [4]:
import math
import sklearn
from sklearn import datasets
from sklearn import svm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# -- Import data --

ts, vs = np.loadtxt("fname", delimiter=',', skiprows=1).transpose()[0, 1]

plt.plot(ts, vs)
plt.show()

In [ ]:
# -- Organise data --

nbits = 56
start = 0.01
end = 0.39

for i, t in enumerate(ts):
    if t < start or t > end:
        del ts[i]
        del vs[i]

intervals = []
for i in range(nbits):
    intervals.append([vs[int(len(vs)/nbits)*i + n] for n in range(int(len(vs)/nbits))])

X = [[np.mean(interval)] for interval in intervals]
y = []

In [2]:
# -- Train machine learning model --

clf = svm.SVC(kernel="linear", C=2)
clf.fit(X, y)



In [ ]:
# -- Identify bits --

bits = clf.predict(X)

In [3]:
# -- Decode ASCII characters --

# Test case
bits = [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]

def decode(bits):
    if len(bits)%7 != 0:
        return "Decoding error 1: Not ASCII sequence."
    bit_strings = []
    for i in range(int(len(bits)/7)):
        bit_string = ''.join([str(bits[7*i + n]) for n in range(7)])
        bit_strings.append(bit_string)
    
    letters = []
    for bitstr in bit_strings:
        num = int('0b' + bitstr, 2)
        letters.append(chr(num))

    return ''.join(letters)
decode(bits)

'NCC-1701'